In [ ]:
# Step 1:
# Mount your Google Drive.
# You will be prompted to authorize access.
from google.colab import drive
drive.mount('/content/drive')

# Define your project's base path in Google Drive for cleanliness.
# IMPORTANT: Make sure you have created this folder in your Drive and uploaded your .zip file there.
# For example, create a folder called "Colab Notebooks" and inside it, a folder called "nemo_asr_project".
PROJECT_PATH = "/content/drive/MyDrive/Colab Notebooks/nemo_asr_project"

# Step 2:
# Install NVIDIA NeMo.
!pip install nemo_toolkit['all']

# Unzip your prepared dataset from Drive into the local Colab session for faster access during training.
# We unzip it to /content/ for speed, but our source is safely in Drive.
!unzip "{PROJECT_PATH}/vn_asr_data_v1.zip" -d /content/data

# Step 3: Load the Pre-trained Model

In [1]:
import nemo.collections.asr as nemo_asr
import os

# --- Step 3: Load the Pre-trained Model ---

# Keep the model name in a variable so later cells can reference it safely.
model_name_ngc = "stt_en_conformer_ctc_large"

# We use the generic 'ASRModel' class. This is the "Universal Loader."
# It automatically detects if the model needs BPE (like Conformer) or not.
print(f"Attempting to load model '{model_name_ngc}' using the Universal Loader...")

asr_model = nemo_asr.models.ASRModel.from_pretrained(model_name=model_name_ngc)

print("Model loaded successfully!")

/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
[NeMo W 2025-12-25 15:28:27 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
      m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
    
[NeMo W 2025-12-25 15:28:27 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
      m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
    
[NeMo W 2025-12-25 15:28:27 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
      elif re.match('(flt)p?( \(default\))?$', token):
    
[NeMo W 2025-12-25 15:28

Attempting to load model 'stt_en_conformer_ctc_large' using the Universal Loader...
[NeMo I 2025-12-25 15:28:28 nemo_logging:393] Downloading from: https://api.ngc.nvidia.com/v2/models/nvidia/nemo/stt_en_conformer_ctc_large/versions/1.10.0/files/stt_en_conformer_ctc_large.nemo to /root/.cache/torch/NeMo/NeMo_2.6.0/stt_en_conformer_ctc_large/afb212c5bcf904e326b5e5751e7c7465/stt_en_conformer_ctc_large.nemo
[NeMo I 2025-12-25 15:28:52 nemo_logging:393] Instantiating model from pre-trained checkpoint
[NeMo I 2025-12-25 15:28:54 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2025-12-25 15:28:54 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath:
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket1/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket2/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket3/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket4/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket5/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket6/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket7/tarred_audio_manifest.json
    - - /data2/nemo_asr/nemo_asr_set_3.0/bucket8/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 1
    shuffle: true
    num_workers: 4
    pin_memory: true
    use_start_end_token: false
    

[NeMo I 2025-12-25 15:28:54 nemo_logging:393] PADDING: 0
[NeMo I 2025-12-25 15:28:56 nemo_logging:393] Model EncDecCTCModelBPE was successfully restored from /root/.cache/torch/NeMo/NeMo_2.6.0/stt_en_conformer_ctc_large/afb212c5bcf904e326b5e5751e7c7465/stt_en_conformer_ctc_large.nemo.
Model loaded successfully!


# Step 4: Verify Data Pipeline

In [ ]:
import json
import random
import os

manifest_path = '/content/data/train_manifest.json'

print(f"Reading manifest from: {manifest_path}")

# Read the manifest file
data_entries = []
with open(manifest_path, 'r') as f:
    for line in f:
        data_entries.append(json.loads(line))

print(f"Successfully loaded {len(data_entries)} samples from the training set.")

# Pick a random sample to inspect
sample = data_entries[0] # Let's look at the first one
audio_path = sample['audio_filepath']
duration = sample['duration']

print("\n=== Data Pipeline Check ===")
print(f"Audio File: {audio_path}")
print(f"Original Duration: {duration:.2f} seconds")
print(f"File Exists: {os.path.exists(audio_path)}")
print("===========================")

Reading manifest from: /content/data/train_manifest.json
Successfully loaded 7 samples from the training set.

=== Data Pipeline Check ===
Audio File: /Users/nguyenhuyvu/nemo-vietnamese-asr/audio/qZuxop5xj_E.wav
Original Duration: 1287.48 seconds
File Exists: False


# Step 5: End-to-End Inference Test (Robust API Check)

In [ ]:
import librosa
import soundfile as sf
import os
import json



print("Preparing audio for inference...")

# 1. GET DATA FROM MANIFEST
# We reuse the 'sample' loaded in Step 4
original_path = sample['audio_filepath']
filename = os.path.basename(original_path) # Extracts 'qZuxop5xj_E.wav'

# 2. CONSTRUCT CORRECT COLAB PATH
colab_audio_path = os.path.join('/content/data/audio', filename)

if not os.path.exists(colab_audio_path):
    raise FileNotFoundError(f"Could not find audio at: {colab_audio_path}")

# 3. LOAD & CROP (First 30s)
y, sr = librosa.load(colab_audio_path, sr=16000, duration=30.0)
temp_path = '/content/temp_30s.wav'
sf.write(temp_path, y, sr)
print(f"Created temporary 30s inference clip: {temp_path}")

# 4. RUN INFERENCE (With API Auto-Fix)
print("Running NVIDIA NeMo Transcribe...")
asr_model.eval()

# Let's inspect what the model expects (for your own knowledge)
print("\n--- Debug: Method Signature ---")
try:
    import inspect
    print(inspect.signature(asr_model.transcribe))
except:
    pass
print("-------------------------------\n")

transcriptions = []

try:
    # Attempt 1: The Standard Way (Keyword Argument)
    print("Attempting with 'paths2audio_files'...")
    transcriptions = asr_model.transcribe(paths2audio_files=[temp_path])
except TypeError:
    try:
        # Attempt 2: Positional Argument (Likely Fix)
        print("Attempting with positional arguments...")
        transcriptions = asr_model.transcribe([temp_path])
    except TypeError:
        # Attempt 3: New 'audio' keyword (Possible 2.0 change)
        print("Attempting with 'audio' keyword...")
        transcriptions = asr_model.transcribe(audio=[temp_path])

# 5. DISPLAY RESULTS
print("\n" + "="*40)
print(f"🤖 MODEL: {model_name_ngc}")
print(f"📂 INPUT: {filename} (First 30s)")
print("-" * 40)
print("🎙️ TRANSCRIPTION:")
if transcriptions:
    print(f"'{transcriptions[0]}'")
else:
    print("ERROR: Could not run inference. Check logs.")
print("="*40)

[NeMo W 2025-12-13 10:30:45 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: use_start_end_token
[NeMo W 2025-12-13 10:30:45 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)


Preparing audio for inference...
Created temporary 30s inference clip: /content/temp_30s.wav
Running NVIDIA NeMo Transcribe...

--- Debug: Method Signature ---
(audio: Union[str, List[str], torch.Tensor, numpy.ndarray, torch.utils.data.dataloader.DataLoader], batch_size: int = 4, return_hypotheses: bool = False, num_workers: int = 0, channel_selector: Union[int, Iterable[int], str, NoneType] = None, augmentor: omegaconf.dictconfig.DictConfig = None, verbose: bool = True, timestamps: Optional[bool] = None, override_config: Optional[nemo.collections.asr.parts.mixins.transcription.TranscribeConfig] = None) -> Union[List[str], List[nemo.collections.asr.parts.utils.rnnt_utils.Hypothesis], Tuple[List[str]], Tuple[List[nemo.collections.asr.parts.utils.rnnt_utils.Hypothesis]]]
-------------------------------

Attempting with 'paths2audio_files'...
Attempting with positional arguments...


Transcribing: 1it [00:01,  1.96s/it]


🤖 MODEL: stt_en_conformer_ctc_large
📂 INPUT: qZuxop5xj_E.wav (First 30s)
----------------------------------------
🎙️ TRANSCRIPTION:
'Hypothesis(score=tensor(-112.7178), y_sequence=tensor([128, 128, 128, 128,  66, 128, 128, 128,   8, 128, 128,  44, 128, 128,
          8, 128,   3, 128, 128, 128, 128, 128, 128, 115, 128, 128, 128, 128,
        128,  25, 128, 128,  47,   8,  18,  30, 128, 128,  31, 128, 128, 128,
        128,  64, 128,  45, 128,  20, 128,   1,  30,  15,   4, 128, 128, 128,
         50,   1, 128,  28, 128,  22, 128,  20,  12, 128, 128,   1, 128,  93,
        128,   5, 128,   9,   6, 128, 128, 128, 128, 128, 128, 128,  63, 128,
        128,   6, 128, 128,   1, 128, 128, 128, 128, 128, 106, 128,  28, 128,
         31, 128, 128, 128, 128, 128,  85, 128, 128,   3, 128,  56, 128, 128,
          3,  35, 128,   4,  42, 128,   1,  14,  28, 128,  22,   1,  11, 128,
        128,  14, 128,  67, 128,   4,  47, 128, 128, 128, 128,  23, 128, 128,
        128,   5, 128,   8, 128, 128,  


# Step 6: Manual Evaluation Loop

In [ ]:
import json
import os
import librosa
import soundfile as sf
import jiwer


print("Starting Manual Evaluation on Test Set...")

test_manifest_path = '/content/data/test_manifest.json'

# 1. Read the Test Manifest
with open(test_manifest_path, 'r') as f:
    test_samples = [json.loads(line) for line in f]

print(f"Found {len(test_samples)} samples in test set.")

# 2. Evaluation Loop
asr_model.eval()

total_wer = 0
count = 0

for sample in test_samples:
    original_path = sample['audio_filepath']
    reference_text = sample['text']
    filename = os.path.basename(original_path)

    colab_path = os.path.join('/content/data/audio', filename)

    if not os.path.exists(colab_path):
        continue

    try:
        # Crop to 30s
        y, sr = librosa.load(colab_path, sr=16000, duration=30.0)
        temp_path = f"/content/temp_eval_{count}.wav"
        sf.write(temp_path, y, sr)

        # Transcribe
        try:
            # Try keyword arg first
            preds = asr_model.transcribe(paths2audio_files=[temp_path])
        except TypeError:
            # Fallback to positional
            preds = asr_model.transcribe([temp_path])

        # --- THE FIX IS HERE ---
        # Check if output is a string or an object
        hypothesis = preds[0]
        if not isinstance(hypothesis, str):
            # It's a Hypothesis object, extract the text
            hypothesis = hypothesis.text

        # Calculate Metrics
        wer = jiwer.wer(reference_text, hypothesis)
        total_wer += wer
        count += 1

        print(f"\n--- Sample {count} ---")
        print(f"Ref: {reference_text[:50]}...")
        print(f"Hyp: {hypothesis[:50]}...")
        print(f"WER: {wer:.2f}")

    except Exception as e:
        print(f"Error processing {filename}: {e}")

if count > 0:
    avg_wer = total_wer / count
    print("\n" + "="*40)
    print(f"Evaluation Complete. Average WER: {avg_wer:.2f}")
    print("="*40)

[NeMo W 2025-12-13 10:35:47 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: use_start_end_token
[NeMo W 2025-12-13 10:35:47 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)


Starting Manual Evaluation on Test Set...
Found 2 samples in test set.


Transcribing: 1it [00:00,  5.20it/s]
[NeMo W 2025-12-13 10:35:48 nemo_logging:405] The following configuration keys are ignored by Lhotse dataloader: use_start_end_token
[NeMo W 2025-12-13 10:35:48 nemo_logging:405] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)



--- Sample 1 ---
Ref: chào bạn cảm ơn bạn đã nhấn nghe giang ngơi radio ...
Hyp: ta bak coming back do onion n ang the radio no mei...
WER: 1.00


Transcribing: 1it [00:00,  7.15it/s]


--- Sample 2 ---
Ref: chào bạn cảm ơn bạn đã nhấn nghe ra hơi radio nơi ...
Hyp: tbak coming back don your nas on the radio no momi...
WER: 1.00

Evaluation Complete. Average WER: 1.00


# Step 7: Save your final model directly to your Google Drive for persistence.

In [ ]:
import os

model_save_path = os.path.join(PROJECT_PATH, "vietnamese_asr_v1.nemo")
print(f"Saving the final model to: {model_save_path}")
asr_model.save_to(model_save_path)
print("Model saved successfully to Google Drive!")

Saving the final model to: /content/drive/MyDrive/Colab Notebooks/nemo_asr_project/vietnamese_asr_v1.nemo
Model saved successfully to Google Drive!


## Step 8: Quantization Benchmark (float32 vs float16 vs int8)

This section benchmarks inference latency and WER across different precision settings on Colab T4.

It runs the repo’s `benchmark.py`, prints a Markdown table, and you can paste the output into the README’s “Production-Ready Model Optimization” section.

In [13]:
# Step 8A: Fetch repo code (so benchmark.py exists) + install benchmark dependencies

# This notebook is intended to be uploaded and run in Colab.
# To avoid any mismatch between your local changes and the GitHub repo state, this cell will:
#  1) Clone the repo (for folder structure)
#  2) Install required packages
#  3) Write stable benchmark + helper modules into the cloned repo
#  4) Sanity-check that they compile

REPO_URL = "https://github.com/wheevu/nemo-vietnamese-asr.git"
REPO_DIR = "/content/nemo-vietnamese-asr"

import os
from pathlib import Path

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL} {REPO_DIR}

%cd {REPO_DIR}

# Ensure dependencies are present (Colab usually includes torch already).
# NeMo is installed in Step 1, but keep a safety install in case you run only Step 8.
!pip -q install "nemo_toolkit[all]"
!pip -q install -r requirements.txt
!pip -q install jiwer "quanto==0.0.11" librosa

# Write helper modules used by benchmark.py
Path("src").mkdir(parents=True, exist_ok=True)

(Path("src") / "__init__.py").write_text(r'''"""Workspace-local utilities."""
''', encoding="utf-8")

(Path("src") / "model_utils.py").write_text(r'''from __future__ import annotations

import os
import warnings
from typing import Optional

import torch


def get_default_device() -> torch.device:
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")


def load_nemo_model(model_ref: str, device: Optional[torch.device] = None):
    device = device or get_default_device()
    import nemo.collections.asr as nemo_asr

    if model_ref.endswith(".nemo") and os.path.exists(model_ref):
        model = nemo_asr.models.ASRModel.restore_from(restore_path=model_ref)
    else:
        model = nemo_asr.models.ASRModel.from_pretrained(model_name=model_ref)

    model.eval()
    model = model.to(device)
    return model


def _quanto_quantize_in_place(model) -> None:
    from quanto import freeze, quantize
    model.eval()
    try:
        from quanto import qint8
        try:
            quantize(model, weights=qint8)
        except TypeError:
            quantize(model, qint8)
    except Exception:
        quantize(model)
    freeze(model)


def _quanto_quantize_linear_layers_in_place(model) -> int:
    import torch.nn as nn
    from quanto import freeze, quantize
    try:
        from quanto import qint8
        q = qint8
    except Exception:
        q = None

    quantized_count = 0
    for _name, module in model.named_modules():
        if not isinstance(module, nn.Linear):
            continue
        try:
            if q is None:
                quantize(module)
            else:
                try:
                    quantize(module, weights=q)
                except TypeError:
                    quantize(module, q)
            quantized_count += 1
        except Exception:
            continue

    if quantized_count > 0:
        freeze(model)
    return quantized_count


def apply_quantization(model, precision: str):
    precision = (precision or "float32").lower().strip()

    if precision == "float32":
        return model

    if precision == "float16":
        return model.half()

    if precision == "int8":
        try:
            _quanto_quantize_in_place(model)
            return model
        except Exception as e:
            warnings.warn(f"Full-model INT8 failed: {e}. Trying Linear-only.", RuntimeWarning)

        try:
            n = _quanto_quantize_linear_layers_in_place(model)
            if n > 0:
                warnings.warn(f"Linear-only INT8 succeeded for {n} layers.", RuntimeWarning)
                return model
        except Exception as e:
            warnings.warn(f"Linear-only INT8 failed: {e}.", RuntimeWarning)

        warnings.warn("Falling back to float32.", RuntimeWarning)
        return model

    raise ValueError(f"Unsupported precision: {precision}")
''', encoding="utf-8")

(Path("src") / "inference.py").write_text(r'''from __future__ import annotations

import inspect
from typing import List, Sequence

import torch


def transcribe_audio(model, audio_files: Sequence[str], batch_size: int = 1) -> List[str]:
    if not audio_files:
        return []

    def _to_text(p) -> str:
        if hasattr(p, "text"):
            try:
                return str(getattr(p, "text"))
            except Exception:
                pass
        return str(p)

    model.eval()
    # Use no_grad instead of inference_mode for Quanto compatibility
    with torch.no_grad():
        transcribe_fn = getattr(model, "transcribe")

        sig = None
        try:
            sig = inspect.signature(transcribe_fn)
        except (TypeError, ValueError):
            sig = None

        if sig is not None and "paths2audio_files" in sig.parameters:
            preds = transcribe_fn(paths2audio_files=list(audio_files), batch_size=batch_size)
        elif sig is not None and "audio" in sig.parameters:
            preds = transcribe_fn(audio=list(audio_files), batch_size=batch_size)
        else:
            try:
                preds = transcribe_fn(list(audio_files), batch_size=batch_size)
            except TypeError:
                preds = transcribe_fn(list(audio_files))

    return [_to_text(p) for p in preds]
''', encoding="utf-8")

# Write benchmark.py with 30s chunking + graceful int8 error handling
Path("benchmark.py").write_text(r'''from __future__ import annotations

import argparse
import hashlib
import io
import json
import os
import tempfile
import time
from dataclasses import dataclass
from typing import Dict, List, Optional, Sequence, Tuple

import torch

from src.inference import transcribe_audio
from src.model_utils import apply_quantization, get_default_device, load_nemo_model


@dataclass(frozen=True)
class ManifestSample:
    audio_filepath: str
    text: str


def read_manifest_samples(manifest_path: str, limit: int) -> List[ManifestSample]:
    samples: List[ManifestSample] = []
    with open(manifest_path, "r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                continue
            obj = json.loads(line)
            if "audio_filepath" not in obj or "text" not in obj:
                continue
            samples.append(
                ManifestSample(audio_filepath=str(obj["audio_filepath"]), text=str(obj["text"]))
            )
            if len(samples) >= limit:
                break
    return samples


def normalize_text(text: str) -> str:
    return " ".join((text or "").strip().lower().split())


def compute_wer_percent(references: Sequence[str], hypotheses: Sequence[str]) -> float:
    import jiwer
    refs = [normalize_text(r) for r in references]
    hyps = [normalize_text(h) for h in hypotheses]
    return float(jiwer.wer(refs, hyps) * 100.0)


def serialized_state_dict_size_mb(model) -> float:
    buf = io.BytesIO()
    cpu_state = {k: v.detach().cpu() for k, v in model.state_dict().items()}
    torch.save(cpu_state, buf)
    return buf.getbuffer().nbytes / (1024 * 1024)


def cuda_sync_if_needed(device: torch.device) -> None:
    if device.type == "cuda":
        torch.cuda.synchronize()


def maybe_remap_audio_path(path: str, audio_root: Optional[str]) -> str:
    if os.path.exists(path) or not audio_root:
        return path
    return os.path.join(audio_root, os.path.basename(path))


MAX_AUDIO_DURATION_S = 30.0
TARGET_SAMPLE_RATE = 16000


def prepare_audio_for_benchmark(path: str) -> str:
    """Prepare audio: mono, 16kHz, max 30s to avoid Conformer OOM."""
    import librosa
    import numpy as np
    import soundfile as sf

    tmp_dir = os.path.join(tempfile.gettempdir(), "nemo_vietnamese_asr_bench")
    os.makedirs(tmp_dir, exist_ok=True)

    info = sf.info(path)
    key_src = f"{os.path.abspath(path)}|{info.samplerate}|{info.channels}|{info.duration}|{MAX_AUDIO_DURATION_S}".encode("utf-8")
    key = hashlib.sha1(key_src).hexdigest()[:12]
    base = os.path.splitext(os.path.basename(path))[0]
    out_path = os.path.join(tmp_dir, f"{base}.prep.{key}.wav")
    if os.path.exists(out_path):
        return out_path

    y, sr = librosa.load(path, sr=TARGET_SAMPLE_RATE, mono=True, duration=MAX_AUDIO_DURATION_S)
    sf.write(out_path, y, sr)
    return out_path


def collect_existing_pairs(
    samples: Sequence[ManifestSample], audio_root: Optional[str]
) -> Tuple[List[str], List[str]]:
    audio_paths: List[str] = []
    references: List[str] = []

    for s in samples:
        p = maybe_remap_audio_path(s.audio_filepath, audio_root)
        if not os.path.exists(p):
            continue
        try:
            p = prepare_audio_for_benchmark(p)
        except Exception:
            continue
        audio_paths.append(p)
        references.append(s.text)

    return audio_paths, references


def benchmark_one_precision(
    *,
    model_ref: str,
    manifest_path: str,
    precision: str,
    sample_limit: int,
    batch_size: int,
    warmup_runs: int,
    device: torch.device,
    audio_root: Optional[str],
) -> Dict[str, object]:
    # Reload model each run (no state leakage)
    try:
        model = load_nemo_model(model_ref, device=device)
        model = apply_quantization(model, precision)
    except Exception as e:
        return {
            "Precision": precision,
            "Avg Latency (ms/file)": None,
            "WER (%)": None,
            "VRAM/Size (MB)": None,
            "Samples": 0,
            "Notes": f"Load/quantization failed: {type(e).__name__}",
        }

    samples = read_manifest_samples(manifest_path, limit=sample_limit)
    audio_files, references = collect_existing_pairs(samples, audio_root)

    model_size_mb = serialized_state_dict_size_mb(model)

    if not audio_files:
        return {
            "Precision": precision,
            "Avg Latency (ms/file)": None,
            "WER (%)": None,
            "VRAM/Size (MB)": round(model_size_mb, 2),
            "Samples": 0,
            "Notes": "No audio files found.",
        }

    warmup_batch = audio_files[: max(1, min(batch_size, len(audio_files)))]
    try:
        with torch.no_grad():
            for _ in range(max(0, warmup_runs)):
                _ = transcribe_audio(model, warmup_batch, batch_size=len(warmup_batch))
    except RuntimeError as e:
        return {
            "Precision": precision,
            "Avg Latency (ms/file)": None,
            "WER (%)": None,
            "VRAM/Size (MB)": round(model_size_mb, 2),
            "Samples": 0,
            "Notes": f"Inference failed (Quanto issue): {type(e).__name__}",
        }

    vram_peak_mb: Optional[float] = None
    if device.type == "cuda":
        torch.cuda.reset_peak_memory_stats()

    hypotheses: List[str] = []
    cuda_sync_if_needed(device)
    t0 = time.perf_counter()

    try:
        with torch.no_grad():
            for i in range(0, len(audio_files), batch_size):
                batch = audio_files[i : i + batch_size]
                hypotheses.extend(transcribe_audio(model, batch, batch_size=len(batch)))
    except RuntimeError as e:
        return {
            "Precision": precision,
            "Avg Latency (ms/file)": None,
            "WER (%)": None,
            "VRAM/Size (MB)": round(model_size_mb, 2),
            "Samples": 0,
            "Notes": f"Inference failed: {type(e).__name__}",
        }

    cuda_sync_if_needed(device)
    t1 = time.perf_counter()

    if device.type == "cuda":
        vram_peak_mb = torch.cuda.max_memory_allocated() / (1024 * 1024)

    total_s = t1 - t0
    avg_latency_ms = (total_s / len(audio_files)) * 1000.0
    wer_percent = compute_wer_percent(references, hypotheses)

    vram_or_size_mb = vram_peak_mb if vram_peak_mb is not None else model_size_mb

    return {
        "Precision": precision,
        "Avg Latency (ms/file)": round(avg_latency_ms, 2),
        "WER (%)": round(wer_percent, 2),
        "VRAM/Size (MB)": round(vram_or_size_mb, 2),
        "Samples": len(audio_files),
    }


def results_to_markdown_table(rows: List[Dict[str, object]]) -> str:
    cols = ["Precision", "Avg Latency (ms/file)", "WER (%)", "VRAM/Size (MB)", "Samples", "Notes"]
    header = "| " + " | ".join(cols) + " |"
    sep = "| " + " | ".join(["---"] * len(cols)) + " |"
    body = ["| " + " | ".join(str(r.get(c, "")) for c in cols) + " |" for r in rows]
    return "\n".join([header, sep] + body)


def main() -> int:
    parser = argparse.ArgumentParser(description="Benchmark NeMo ASR quantization.")
    parser.add_argument("--model", required=True)
    parser.add_argument("--manifest", default="val_manifest.json")
    parser.add_argument("--samples", type=int, default=100)
    parser.add_argument("--batch-size", type=int, default=1)
    parser.add_argument("--warmup-runs", dest="warmup_runs", type=int, default=3)
    parser.add_argument("--warmup-batches", dest="warmup_runs", type=int)
    parser.add_argument("--audio-root", default=None)

    args = parser.parse_args()

    device = get_default_device()
    precisions = ["float32", "float16", "int8"]

    results: List[Dict[str, object]] = []
    for p in precisions:
        print(f"\n>>> Benchmarking {p}...")
        results.append(
            benchmark_one_precision(
                model_ref=args.model,
                manifest_path=args.manifest,
                precision=p,
                sample_limit=max(1, args.samples),
                batch_size=max(1, args.batch_size),
                warmup_runs=max(0, args.warmup_runs or 0),
                device=device,
                audio_root=args.audio_root,
            )
        )

    print("\n=== Benchmark Results (raw) ===")
    for r in results:
        print(r)

    print("\n=== Markdown Table (paste into README) ===")
    print(results_to_markdown_table(results))

    return 0


if __name__ == "__main__":
    raise SystemExit(main())
''', encoding="utf-8")

# Sanity check: compile the generated files
!python -m py_compile benchmark.py src/model_utils.py src/inference.py

print("Repo ready for benchmarking:", os.getcwd())
print("Wrote updated benchmark + helper modules.")

/content/nemo-vietnamese-asr
Repo ready for benchmarking: /content/nemo-vietnamese-asr
Wrote updated benchmark + helper modules.


In [14]:
# Step 8B: Run benchmark.py (prints a Markdown table for README)

# Choose which model to benchmark:
import os

# Prefer your trained NeMo artifact if it exists; otherwise fall back to the NGC model name.
DEFAULT_NGC = globals().get("model_name_ngc", "stt_en_conformer_ctc_large")
SAVED_NEMO = os.path.join(PROJECT_PATH, "vietnamese_asr_v1.nemo")
MODEL_REF = SAVED_NEMO if os.path.exists(SAVED_NEMO) else DEFAULT_NGC
print("Benchmarking model:", MODEL_REF)

# Use whichever manifest you want to evaluate on; start with test to keep it small.
MANIFEST_PATH = "/content/data/test_manifest.json"

# Remap macOS absolute paths by basename into Colab audio dir (matches your notebook's path-bridging approach).
AUDIO_ROOT = "/content/data/audio"

# Keep runtime reasonable on Colab
SAMPLES = 100
BATCH_SIZE = 1
WARMUP_RUNS = 3

!python benchmark.py --model "{MODEL_REF}" --manifest "{MANIFEST_PATH}" --samples {SAMPLES} --batch-size {BATCH_SIZE} --warmup-runs {WARMUP_RUNS} --audio-root "{AUDIO_ROOT}"

Benchmarking model: /content/drive/MyDrive/Colab Notebooks/nemo_asr_project/vietnamese_asr_v1.nemo
/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]

>>> Benchmarking float32...
fused_indices_to_multihot has reached end of life. Please migrate to a non-experimental function.
2025-12-25 16:03:54.480746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766678634.501492   13257 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766678634.507728   13257 cuda_blas.cc:1407] Unable to regis